# Commo VaR / Sensi

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from xbbg import blp

In [2]:
Start = "20180101"
End = ""

SPX = blp.bdh("SPX Index", "PX_Last", start_date=Start)
SPX.columns = ["SPX"]


|Conf. Interval | Z-Score|
-----|-------
|    90%  | 1.645 |
|    95%  | 1.96  |
|    99%  |	2.576 |


# Pick a Commo and a contract:

In [93]:
Commo = ["GCQ0", "SIQ0"]

In [102]:
Pointer = blp.bdh([x + " Comdty" for x in Commo], "PX_Last", start_date=Start)
Pointer.columns = Commo
Pointer = (Pointer / Pointer.shift(1) - 1).dropna()

In [103]:
Pointer 

,GCQ0,SIQ0
2018-09-27,1247.1,NaN
2018-09-28,1256.0,NaN
2018-10-01,1251.7,NaN
2018-10-02,1267.4,NaN
2018-10-03,1263.4,NaN
...,...,...
2020-06-17,1735.6,17.860
2020-06-18,1731.1,17.601
2020-06-19,1753.0,17.937
2020-06-22,1766.4,17.992


In [113]:
1000 * Pointer.iloc[-252:].std()[0] * 1.96

21.512919445016173

In [98]:
CIdico = {0.95:1.96,
         0.9:1.645,
         0.99:2.576}


In [105]:
def VaR(df, ConfInt = 0.95, Investment = 3000000):
    Weight = [1/len(df.T) for x in range(len(df.T))]
    Sigma = df.iloc[-252:].cov()
    return CIdico[ConfInt] * Investment * np.sqrt(np.array(Weight) @ Sigma @ np.array(Weight).T) 
    


In [110]:
VaR(Pointer)

26.921775203181618

In [119]:
Pointer.iloc[-252:].cov()

,GCQ0,SIQ0
GCQ0,0.000120,0.000155
SIQ0,0.000155,0.000325


In [118]:
Pointer.iloc[-252:].corr()

,GCQ0,SIQ0
GCQ0,1.000000,0.780626
SIQ0,0.780626,1.000000


In [120]:
test = Pointer.iloc[-252:].cov()

In [121]:
test["VaR"] = VaR(Pointer)

In [123]:
test.to_csv("testVaR.csv")

In [124]:
ls

 Volume in drive C has no label.
 Volume Serial Number is BCF2-6F03

 Directory of C:\Users\HanaDT\Projects\Strategies\Quantitative\Commo VaR-Sensi

23/06/2020  15:46    <DIR>          .
23/06/2020  15:46    <DIR>          ..
22/06/2020  14:45    <DIR>          .ipynb_checkpoints
23/06/2020  15:46               157 testVaR.csv
23/06/2020  15:46            10,022 VaR.ipynb
               2 File(s)         10,179 bytes
               3 Dir(s)  101,544,050,688 bytes free


In [132]:
tes = pd.read_clipboard()

In [135]:
tes.std()**2


x             0.000449
Unnamed: 1    0.000141
dtype: float64

In [138]:
(np.array(tes).T @ np.array(tes)) / len(tes)

array([[0.00044851, 0.00019116],
       [0.00019116, 0.00014214]])